# DATA SCIENCE INTERNSHIP @ OASIS INFOBYTE SIP (OIBSIP)

### AUTHOR : PARVEJ ALAM M. ANSARI

# TASK 4 : Email Spam/Ham Detection With Machine Learning

### The dataset is available at Kaggle : 

### https://www.kaggle.com/datasets/shantanudhakadd/email-spam-detection-dataset-classification

### Context:

- Classifying emails into distinct labels can have a great impact on customer support. By using machine learning to label emails the system can set up queues containing emails of a specific category. This enables support personnel to handle request quicker and more easily by selecting a queue that match their expertise.

### Objectives:

- This study aims to improve the manually defined rule based algorithm, currently implemented at a large telecom company, by using machine learning. The proposed model should have higher F1-score and classification rate. Integrating or migrating from a manually defined rule based model to a machine learning model should also reduce the administrative and maintenance work. It should also make the model more flexible

In [1]:
from IPython.display import Image
Image(url='https://miro.medium.com/v2/resize:fit:720/format:webp/1*nBgCTU_hAVG00eYkcRf6Mw.png', height=100, width=400)

## 1. Import Required Libraries:

In [3]:
# Import python liabraries from scikit-learn.

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,f1_score,recall_score,precision_score


In [4]:
# Load dataset:
df = pd.read_csv("spam.csv", encoding="latin1")
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


## 2. Exploratory Data Analysis (EDA):

In [5]:
# Check columns in dataframe:
df.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [6]:
# Check descriptive statistics
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [7]:
# Check the number of rows and columns present in df:
print('Number of Rows:',df.shape[0])
print('Number of Columns:',df.shape[1])

Number of Rows: 5572
Number of Columns: 5


In [8]:
# Null value count in dataframe:
df.isnull().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [9]:
df.isnull().mean()*100  # Check the percentage of null value

v1             0.000000
v2             0.000000
Unnamed: 2    99.102656
Unnamed: 3    99.784637
Unnamed: 4    99.892319
dtype: float64

#### As we can see there are huge number of missing entries in Unnamed:2,Unnamed:3,Unnamed:4 col i.e more than 99%.So we should have to remove these column.

In [10]:
df.drop(columns=df[['Unnamed: 2','Unnamed: 3','Unnamed: 4']],axis=1,inplace=True)

In [11]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [12]:
df.shape

(5572, 2)

In [13]:
# Rename columns names for easy to understand, we can also use df.rename
df.columns=['spam/ham','sms']

In [14]:
# Convert the text data into numerical form
df.loc[df['spam/ham'] == 'spam', 'spam/ham',] = 0
df.loc[df['spam/ham'] == 'ham', 'spam/ham',] = 1

In [15]:
df.head()

,spam/ham,sms
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."


## 3. Separating Input Features and Target Column:

In [16]:
x = df.sms
x.head()

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
Name: sms, dtype: object

In [17]:
y = df['spam/ham']
y.head()

0    1
1    1
2    0
3    1
4    1
Name: spam/ham, dtype: object

## 4. Train-Test-Split: 

In [18]:
# Divide the whole dataset into training and testing set for model training
from sklearn.model_selection import train_test_split

In [19]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2,random_state=42)

In [20]:
# Checking shape of train and test dataset:
print(x.shape)
print(xtrain.shape)
print(xtest.shape)

(5572,)
(4457,)
(1115,)


In [21]:
xtrain,xtest

(1978    No I'm in the same boat. Still here at my moms...
 3989    (Bank of Granite issues Strong-Buy) EXPLOSIVE ...
 3935       They r giving a second chance to rahul dengra.
 4078       O i played smash bros  &lt;#&gt;  religiously.
 4086    PRIVATE! Your 2003 Account Statement for 07973...
                               ...                        
 3772    I came hostel. I m going to sleep. Plz call me...
 5191                               Sorry, I'll call later
 5226        Prabha..i'm soryda..realy..frm heart i'm sory
 5390                           Nt joking seriously i told
 860                   In work now. Going have in few min.
 Name: sms, Length: 4457, dtype: object,
 3245    Funny fact Nobody teaches volcanoes 2 erupt, t...
 944     I sent my scores to sophas and i had to do sec...
 1044    We know someone who you know that fancies you....
 2484    Only if you promise your getting out as SOON a...
 812     Congratulations ur awarded either å£500 of CD ...
               

In [22]:
ytrain,ytest

(1978    1
 3989    0
 3935    1
 4078    1
 4086    0
        ..
 3772    1
 5191    1
 5226    1
 5390    1
 860     1
 Name: spam/ham, Length: 4457, dtype: object,
 3245    1
 944     1
 1044    0
 2484    1
 812     0
        ..
 4264    1
 2439    1
 5556    1
 4205    1
 4293    0
 Name: spam/ham, Length: 1115, dtype: object)

## 5. Text to Vector Conversion:

#### As we know that,machine learning algorithms only performs well with respect to numbers,so we need to convert all the text data into numbers.To do so I will use TfidfVectorizer techinque from feature_extraction of sklearn.

In [23]:
feat_vect = TfidfVectorizer(min_df=1, stop_words='english', lowercase=True)
feat_vect

TfidfVectorizer(stop_words='english')

In [24]:
ytrain = ytrain.astype('int')
ytest = ytest.astype('int')

In [25]:
xtrain_vec = feat_vect.fit_transform(xtrain)

In [26]:
xtest_vec = feat_vect.transform(xtest)

In [27]:
print(xtrain)

1978    No I'm in the same boat. Still here at my moms...
3989    (Bank of Granite issues Strong-Buy) EXPLOSIVE ...
3935       They r giving a second chance to rahul dengra.
4078       O i played smash bros  &lt;#&gt;  religiously.
4086    PRIVATE! Your 2003 Account Statement for 07973...
                              ...                        
3772    I came hostel. I m going to sleep. Plz call me...
5191                               Sorry, I'll call later
5226        Prabha..i'm soryda..realy..frm heart i'm sory
5390                           Nt joking seriously i told
860                   In work now. Going have in few min.
Name: sms, Length: 4457, dtype: object


In [28]:
xtrain_vec

<4457x7472 sparse matrix of type '<class 'numpy.float64'>'
	with 34794 stored elements in Compressed Sparse Row format>

In [29]:
print(xtrain_vec)

  (0, 4520)	0.4658046386365619
  (0, 3210)	0.348722265231364
  (0, 7415)	0.348722265231364
  (0, 1706)	0.3431839629173582
  (0, 4416)	0.4528381701109944
  (0, 1371)	0.4658046386365619
  (1, 0)	0.2654936554684193
  (1, 1649)	0.3059746053542906
  (1, 6440)	0.2953742837684993
  (1, 4533)	0.3059746053542906
  (1, 419)	0.28715203556385105
  (1, 4292)	0.2953742837684993
  (1, 5005)	0.1937920260229529
  (1, 2661)	0.3059746053542906
  (1, 1533)	0.2015782058421696
  (1, 6296)	0.269833648032668
  (1, 3631)	0.2804339696184593
  (1, 3140)	0.3059746053542906
  (1, 1187)	0.26161139982801973
  (2, 2190)	0.5102109014477275
  (2, 5351)	0.5102109014477275
  (2, 1674)	0.35156722029872034
  (2, 5770)	0.3962151014046925
  (2, 3061)	0.44585171875646595
  (3, 5484)	0.4829129976175997
  :	:
  (4451, 5740)	0.3358090891373877
  (4451, 4686)	0.3478605253385091
  (4452, 3402)	0.4536077050510107
  (4452, 3423)	0.4833413012939851
  (4452, 1579)	0.3576443319642905
  (4452, 1781)	0.3311324953642251
  (4452, 5998)	0.3

In [30]:
print(xtest_vec)

  (0, 7229)	0.2947064107791228
  (0, 6816)	0.4006242977875035
  (0, 4543)	0.38197308370768035
  (0, 3752)	0.1718556592061185
  (0, 3457)	0.3500886226408095
  (0, 3239)	0.34299776014114036
  (0, 2974)	0.34299776014114036
  (0, 2679)	0.3500886226408095
  (0, 1756)	0.31111329907426943
  (1, 6607)	0.27039238853977376
  (1, 6604)	0.19484478334547534
  (1, 5812)	0.22078293973996208
  (1, 5744)	0.35520030142077386
  (1, 5739)	0.35520030142077386
  (1, 5738)	0.25559165628741076
  (1, 5532)	0.33866381848750327
  (1, 4760)	0.29866169283344046
  (1, 3716)	0.3178303138520559
  (1, 2651)	0.3269309971271071
  (1, 1970)	0.2461378627103295
  (1, 1934)	0.22392171769600464
  (2, 5075)	0.4020147546075029
  (2, 4106)	0.5120683436791947
  (2, 3835)	0.4855870501823454
  (2, 2707)	0.4882288103453305
  :	:
  (1110, 4022)	0.191596066847086
  (1110, 3363)	0.3742235014841453
  (1110, 3174)	0.19113154928290435
  (1110, 2651)	0.3742235014841453
  (1110, 2148)	0.301483654608874
  (1110, 1869)	0.3876535449194833
  (

## 6. Logistic Regression Algorithm:

In [31]:
lr = LogisticRegression()

In [32]:
lr.fit(xtrain_vec,ytrain)

LogisticRegression()

In [33]:
lr.score(xtrain_vec,ytrain)

0.9694862014808167

In [34]:
lr.score(xtest_vec,ytest)

0.9524663677130045

In [35]:
pred_lr=lr.predict(xtest_vec)
pred_lr

array([1, 1, 1, ..., 1, 1, 1])

## 7. Evaluation of Algorithm:

In [36]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [37]:
accuracy_score(ytest,pred_lr)

0.9524663677130045

In [38]:
confusion_matrix(ytest,pred_lr)

array([[100,  50],
       [  3, 962]])

In [39]:
classification_report(ytest,pred_lr)

'              precision    recall  f1-score   support\n\n           0       0.97      0.67      0.79       150\n           1       0.95      1.00      0.97       965\n\n    accuracy                           0.95      1115\n   macro avg       0.96      0.83      0.88      1115\nweighted avg       0.95      0.95      0.95      1115\n'

### Concluding Remarks:
- As we saw, we used previously collected data in order to train the model and predicted the category for new incoming emails. This indicate the importance of tagging the data in right way. 
- One mistake can make your machine dumb, e.g In your gmail or any other email account when you get the emails and you think it is a spam but you choose to ignore, may be next time when you see that email, you should report that as a spam. 
- This process can help a lot of other people who are receiving the same kind of email but not aware of what spam is. Sometimes wrong spam tag can move a genuine email to spam folder too. 
- So, you have to be careful before you tag an email as a spam or not spam.